In [1]:

!pip install -q streamlit
!pip install -q transformers
!pip install -q torch
!pip install -q sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.5 MB/s eta 0:00:00


In [7]:
!pip install -q pyngrok


In [2]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

st.set_page_config(page_title="Flan-T5 Chatbot", page_icon="🤖")

@st.cache_resource
def load_model():
    try:
        model_name = "google/flan-t5-base"


        progress_text = "Model is loading."

        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            low_cpu_mem_usage=True
        )


        if torch.cuda.is_available():
            model = model.to('cuda')

        return tokenizer, model, None
    except Exception as e:
        return None, None, str(e)

with st.spinner('🔄 '):
    tokenizer, model, error = load_model()

if error:
    st.error(f"Model error {error}")
    st.stop()

def chatbot_response(prompt):
    try:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)

        if device == 'cuda':
            inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_length=150,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.9,
                no_repeat_ngram_size=2,
                early_stopping=True
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response
    except Exception as e:
        return f"Hata oluştu: {str(e)}"

# ---- Streamlit App UI ----
st.title("🤖 Flan-T5 Chatbot")
st.caption("Powered by Google Flan-T5")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Chat input
if prompt := st.chat_input("Mesajınızı yazın..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        with st.spinner("Düşünüyor..."):
            response = chatbot_response(prompt)
            st.markdown(response)

    st.session_state.messages.append({"role": "assistant", "content": response})

Writing app.py


In [12]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.81.121.15

In [9]:
!streamlit cache clear


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙

⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.81.121.15:8501

your url is: https://dark-mangos-judge.loca.lt
`torch_dtype` is deprecated! Use `dtype` instead!
2025-11-05 20:12:20.451931: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762373540.509421    5458 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762373540.528666    5458 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762373540.579493    5458 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same